In [29]:
!conda install -c anaconda xlrd --yes
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import xlrd
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0rc0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 anaconda
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0 anaconda


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version

In [7]:
neighborhoods = pd.read_excel(open('geodir-ubigeo-departamento.xlsx', 'rb'),sheet_name='ubigeo_inei') 

print('Data downloaded and read into a dataframe!')
neighborhoods

Data downloaded and read into a dataframe!


,Borough,Neighborhood,Latitude,Longitude
0,2534,Amazonas,-6.231690,-77.869034
1,2625,Ancash - Huaraz,-9.526000,-77.529000
2,2812,Ancash - Chimbote,-9.063000,-78.589000
3,2900,Apurimac,-14.050000,-73.088000
4,3020,Arequipa,-16.398867,-71.536961
5,3143,Ayacucho,-13.160427,-74.225697
6,3292,Cajamarca,-7.162000,-78.513000
7,3414,Cusco,-13.522839,-71.967627
8,3518,Huancavelica,-12.787280,-74.973095
9,3606,Huanuco,-9.929325,-76.239485


In [8]:
address = 'Huancayo, Peru'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lima-Peru are {}, {}.'.format(latitude, longitude)) 

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        
        fill=True,
        fill_color='#3186cc',
        line_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of Lima-Peru are -12.0761679, -75.2063189.


In [11]:
CLIENT_ID = 'UT1ZMPQ4CNFSEZ4XU51VH331ADXRGYF1KWVHIWY4GS2J0OVV' # your Foursquare ID
CLIENT_SECRET = 'GF0HVMAMOPS1URJINQBCPXVO4OVBDMGNKPTMQG32WF0B4B1D' # your Foursquare Secret
VERSION = '20181231'


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhoods.loc[0, 'Neighborhood']
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# type your answer here
CLIENT_ID = 'UT1ZMPQ4CNFSEZ4XU51VH331ADXRGYF1KWVHIWY4GS2J0OVV' # your Foursquare ID
CLIENT_SECRET = 'GF0HVMAMOPS1URJINQBCPXVO4OVBDMGNKPTMQG32WF0B4B1D' # your Foursquare Secret
VERSION = '20181231'
neighborhood_latitude=-12.0623905
neighborhood_longitude=-77.1286283
radius=2500
LIMIT = 30
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
#results

Your credentails:
CLIENT_ID: UT1ZMPQ4CNFSEZ4XU51VH331ADXRGYF1KWVHIWY4GS2J0OVV
CLIENT_SECRET:GF0HVMAMOPS1URJINQBCPXVO4OVBDMGNKPTMQG32WF0B4B1D
Latitude and longitude values of Amazonas are -6.2316899, -77.8690338.


In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues

30 venues were returned by Foursquare.


,name,categories,lat,lng
0,Chifa El Osito,Chinese Restaurant,-12.059813,-77.135372
1,Francesco,Seafood Restaurant,-12.058225,-77.130595
2,Villa Deportiva Regional del Callao,Baseball Field,-12.062383,-77.121090
3,Club de Tiro de Bellavista,Event Space,-12.060454,-77.126604
4,Panadería Olcese,Bakery,-12.061041,-77.143544
5,D' Carbón Pollos & Parrillas,BBQ Joint,-12.068036,-77.114188
6,Casa Fugaz,Art Gallery,-12.060145,-77.147335
7,Lemons gym,Gym,-12.059705,-77.133716
8,CasaCor Callao,Public Art,-12.060004,-77.147282
9,Monumental Callao,Art Gallery,-12.059839,-77.147090


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [15]:
manhattan_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

print(manhattan_venues.shape)
manhattan_venues.head()

print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))
manhattan_venues.groupby('Neighborhood').count()

Amazonas
Ancash - Huaraz
Ancash - Chimbote
Apurimac
Arequipa
Ayacucho
Cajamarca
Cusco
Huancavelica
Huanuco
Ica
Junin
La Libertad
Lambayeque
Lima
Loreto
Madre de Dios
Moquegua
Pasco
Piura
Puno
San Martin
Tacna
Tumbes
Ucayali
Callao
(367, 7)
There are 89 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amazonas,8,8,8,8,8,8
Ancash - Chimbote,6,6,6,6,6,6
Ancash - Huaraz,24,24,24,24,24,24
Arequipa,30,30,30,30,30,30
Ayacucho,15,15,15,15,15,15
Cajamarca,8,8,8,8,8,8
Callao,5,5,5,5,5,5
Cusco,8,8,8,8,8,8
Huancavelica,4,4,4,4,4,4


In [18]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

manhattan_onehot.shape
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped
manhattan_grouped.shape

(25, 90)

In [19]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amazonas----
        venue  freq
0        Café  0.25
1       Plaza  0.12
2        Park  0.12
3  Restaurant  0.12
4         Bar  0.12


----Ancash - Chimbote----
                   venue  freq
0                    Zoo  0.17
1  Outdoors & Recreation  0.17
2   Himalayan Restaurant  0.17
3        Harbor / Marina  0.17
4         Breakfast Spot  0.17


----Ancash - Huaraz----
         venue  freq
0  Pizza Place  0.12
1         Café  0.08
2        Hotel  0.08
3          Bar  0.08
4   Restaurant  0.08


----Arequipa----
                 venue  freq
0  Peruvian Restaurant  0.13
1       Sandwich Place  0.10
2       Chocolate Shop  0.07
3                Hotel  0.07
4                 Café  0.07


----Ayacucho----
         venue  freq
0   Restaurant  0.20
1        Plaza  0.13
2         Café  0.07
3       Market  0.07
4  Pizza Place  0.07


----Cajamarca----
                 venue  freq
0                Plaza  0.25
1           Steakhouse  0.12
2          Coffee Shop  0.12
3          Bus Station 

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amazonas,Café,Park,Peruvian Restaurant,Plaza,Bar,Bed & Breakfast,Restaurant,Farmers Market,Department Store,Dessert Shop
1,Ancash - Chimbote,Zoo,Breakfast Spot,Harbor / Marina,Outdoors & Recreation,Seafood Restaurant,Himalayan Restaurant,Bakery,Flea Market,Diner,Doctor's Office
2,Ancash - Huaraz,Pizza Place,Hotel,Bar,Restaurant,Café,Hostel,Farmers Market,Museum,Nightclub,Mexican Restaurant
3,Arequipa,Peruvian Restaurant,Sandwich Place,Bed & Breakfast,Hotel,Chocolate Shop,Museum,Café,Latin American Restaurant,Pub,Monastery
4,Ayacucho,Restaurant,Plaza,Market,Café,Shopping Mall,Flea Market,Breakfast Spot,Pizza Place,Peruvian Restaurant,Bar


In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged =manhattan_merged.dropna()  # check the last columns!
manhattan_merged['Cluster Labels'] = manhattan_merged['Cluster Labels'].astype(int)

manhattan_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2534,Amazonas,-6.231690,-77.869034,3,Café,Park,Peruvian Restaurant,Plaza,Bar,Bed & Breakfast,Restaurant,Farmers Market,Department Store,Dessert Shop
1,2625,Ancash - Huaraz,-9.526000,-77.529000,3,Pizza Place,Hotel,Bar,Restaurant,Café,Hostel,Farmers Market,Museum,Nightclub,Mexican Restaurant
2,2812,Ancash - Chimbote,-9.063000,-78.589000,3,Zoo,Breakfast Spot,Harbor / Marina,Outdoors & Recreation,Seafood Restaurant,Himalayan Restaurant,Bakery,Flea Market,Diner,Doctor's Office
4,3020,Arequipa,-16.398867,-71.536961,3,Peruvian Restaurant,Sandwich Place,Bed & Breakfast,Hotel,Chocolate Shop,Museum,Café,Latin American Restaurant,Pub,Monastery
5,3143,Ayacucho,-13.160427,-74.225697,3,Restaurant,Plaza,Market,Café,Shopping Mall,Flea Market,Breakfast Spot,Pizza Place,Peruvian Restaurant,Bar
6,3292,Cajamarca,-7.162000,-78.513000,3,Plaza,Coffee Shop,Pizza Place,Fried Chicken Joint,Steakhouse,Japanese Restaurant,Bus Station,Zoo,Farmers Market,Dessert Shop
7,3414,Cusco,-13.522839,-71.967627,3,Train Station,Hotel,Pizza Place,Bed & Breakfast,Plaza,Soccer Stadium,Market,Seafood Restaurant,Zoo,Farmers Market
8,3518,Huancavelica,-12.787280,-74.973095,1,Park,Hotel,BBQ Joint,Plaza,Fish & Chips Shop,Department Store,Dessert Shop,Diner,Doctor's Office,Donut Shop
9,3606,Huanuco,-9.929325,-76.239485,1,Hotel,Pizza Place,Fish & Chips Shop,Café,Gym,Plaza,Coffee Shop,Brewery,Chinese Restaurant,Diner
10,3655,Ica,-14.068000,-75.729000,3,Fast Food Restaurant,Restaurant,Department Store,Hotel,Bookstore,Sandwich Place,Snack Place,Plaza,Café,Italian Restaurant


In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)
       
map_clusters

In [45]:
df_peru = pd.read_excel(open('geodir-ubigeo-peru.xlsx', 'rb'),sheet_name='ubigeo_inei') 

print('Data downloaded and read into a dataframe!')

df_peru['Distrito'] = df_peru.Distrito.astype(str)
df_peru['Provincia'] = df_peru.Provincia.astype(str)

#Group by Departament
df_peru_data=df_peru.groupby('Departamento',as_index=False).sum()
df_peru_data=df_peru_data.apply(lambda x: x.astype(str).str.upper())

#Convert field to float
df_peru_data['Poblacion'] = df_peru_data.Poblacion.astype(float)
df_peru_data['Superficie'] = df_peru_data.Superficie.astype(float)
df_peru_data['Y'] = df_peru_data.Y.astype(float)
df_peru_data['X'] = df_peru_data.X.astype(float)

#Calculate Poblation per Kilometer
df_peru_data['PoblacionxKm2']=df_peru_data['Poblacion']/df_peru_data['Superficie']

#Exclude Lima and Callao Departament to analyses other Department.
df_peru_data=df_peru_data.drop(df_peru_data.index[14])
df_peru_data=df_peru_data.drop(df_peru_data.index[6])
df_peru_data

Data downloaded and read into a dataframe!


,Departamento,Ubigeo,Poblacion,Superficie,Y,X,PoblacionxKm2
0,AMAZONAS,872418,425832.0,39249.13,-513.09100,-6546.49210,10.849463
1,ANCASH,3500411,1166180.0,35881.30,-1547.78840,-12878.17160,32.501052
2,APURIMAC,2553828,464583.0,21037.99,-1169.15150,-6139.68930,22.083051
3,AREQUIPA,4402800,1329801.0,63343.95,-1740.19090,-7886.44640,20.993339
4,AYACUCHO,6022013,711058.0,44357.00,-1653.16360,-8804.64560,16.030345
5,CAJAMARCA,7705419,1540009.0,33304.32,-840.59490,-9994.51770,46.240518
7,CUSCO,9041166,1338897.0,82694.66,-1535.97104,-8051.65712,16.190852
8,HUANCAVELICA,9042953,505495.0,22159.22,-1286.26270,-7491.87530,22.811949
9,HUANUCO,8446193,878196.0,37867.36,-807.24810,-6417.32250,23.191371
10,ICA,4741337,810214.0,21305.51,-600.35930,-3256.05840,38.028379


In [48]:
peru_geo = r'peru_departamental_simple.geojson'

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_peru_data['Poblacion'].min(),
                              df_peru_data['Poblacion'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
peru_map = folium.Map(location=[-12,-77], zoom_start=5, tiles='Mapbox Bright')

map_clusters.choropleth(
    geo_data=peru_geo,
    data=df_peru_data,
    columns=['Departamento', 'Poblacion'],
    key_on='feature.properties.NOMBDEP',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population',
    reset=True
)

map_clusters

In [50]:

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="black",
        #color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1
        ).add_to(map_clusters)

map_clusters

In [24]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Loreto,Concert Hall,Gym / Fitness Center,Gym,Supermarket,Chinese Restaurant,Fish & Chips Shop,Dessert Shop,Diner,Doctor's Office,Donut Shop


In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Huancavelica,Park,Hotel,BBQ Joint,Plaza,Fish & Chips Shop,Department Store,Dessert Shop,Diner,Doctor's Office,Donut Shop
9,Huanuco,Hotel,Pizza Place,Fish & Chips Shop,Café,Gym,Plaza,Coffee Shop,Brewery,Chinese Restaurant,Diner
11,Junin,Café,Hotel,Peruvian Restaurant,Art Gallery,Diner,BBQ Joint,Coffee Shop,Plaza,Restaurant,Zoo
16,Madre de Dios,Video Store,Bus Station,Hotel,Harbor / Marina,Bakery,Fried Chicken Joint,Diner,Doctor's Office,Donut Shop,Electronics Store
18,Pasco,Hotel,BBQ Joint,Fried Chicken Joint,Restaurant,Zoo,Fish & Chips Shop,Dessert Shop,Diner,Doctor's Office,Donut Shop
21,San Martin,Peruvian Restaurant,Hotel,Restaurant,South American Restaurant,Zoo,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Doctor's Office
22,Tacna,Hotel,Plaza,Pizza Place,Bar,Chinese Restaurant,Italian Restaurant,Shopping Mall,Diner,Park,Farmers Market
23,Tumbes,Hotel,Seafood Restaurant,Snack Place,BBQ Joint,Plaza,Café,Dessert Shop,Diner,Doctor's Office,Donut Shop


In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Ucayali,Peruvian Restaurant,Asian Restaurant,Bar,Flea Market,Dessert Shop,Diner,Doctor's Office,Donut Shop,Electronics Store,Farmers Market


In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amazonas,Café,Park,Peruvian Restaurant,Plaza,Bar,Bed & Breakfast,Restaurant,Farmers Market,Department Store,Dessert Shop
1,Ancash - Huaraz,Pizza Place,Hotel,Bar,Restaurant,Café,Hostel,Farmers Market,Museum,Nightclub,Mexican Restaurant
2,Ancash - Chimbote,Zoo,Breakfast Spot,Harbor / Marina,Outdoors & Recreation,Seafood Restaurant,Himalayan Restaurant,Bakery,Flea Market,Diner,Doctor's Office
4,Arequipa,Peruvian Restaurant,Sandwich Place,Bed & Breakfast,Hotel,Chocolate Shop,Museum,Café,Latin American Restaurant,Pub,Monastery
5,Ayacucho,Restaurant,Plaza,Market,Café,Shopping Mall,Flea Market,Breakfast Spot,Pizza Place,Peruvian Restaurant,Bar
6,Cajamarca,Plaza,Coffee Shop,Pizza Place,Fried Chicken Joint,Steakhouse,Japanese Restaurant,Bus Station,Zoo,Farmers Market,Dessert Shop
7,Cusco,Train Station,Hotel,Pizza Place,Bed & Breakfast,Plaza,Soccer Stadium,Market,Seafood Restaurant,Zoo,Farmers Market
10,Ica,Fast Food Restaurant,Restaurant,Department Store,Hotel,Bookstore,Sandwich Place,Snack Place,Plaza,Café,Italian Restaurant
12,La Libertad,Peruvian Restaurant,Tea Room,Café,Bar,Breakfast Spot,Nightclub,Sandwich Place,Convenience Store,Juice Bar,Ice Cream Shop
13,Lambayeque,Peruvian Restaurant,Restaurant,Coffee Shop,Pizza Place,Plaza,Hotel,Supermarket,Bakery,Market,Chinese Restaurant


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
